In [ ]:
import simulation as sim
from importlib import reload
import numpy as np
import pickle
import random
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd
import os
from scipy import stats
from scipy.stats import skewnorm
from scipy.optimize import minimize_scalar
from scipy.stats import skewnorm

In [2]:
# trials = pd.read_pickle('trials_13_Optimising_with_initial_probing.pkl')
# runs = pd.DataFrame(trials.vals)
# runs['loss'] = trials.losses()
# runs.sort_values('loss', inplace=True)

In [5]:
runs[runs.loss < -0.5][['A_eq_star_scaling', 'P_star', 'alpha', 'beta']].describe().loc[['min', 'max']]

,A_eq_star_scaling,P_star,alpha,beta
min,0.139960,674.664022,0.003897,0.059233
max,0.230439,805.191377,0.003943,0.668773


In [13]:
def instantiate_and_run_simulation(
    run_number
):
    simulation = sim.Simulation(N_bodies=50, timestep_reset=1000)

    # beta = random.uniform(100, 150)
    # alpha =  random.uniform(500, 550)
    # A_eq_star_scaling = random.uniform(500, 500)
    # P_star = random.uniform(460.438517, 904.892311) 
    # radius_scaling = random.uniform(0.5, 1.2)
    # volume_scaling = 0.8

    beta = 45.687782
    alpha =  0.329671
    A_eq_star_scaling = 0.448194
    P_star = 0.886839 
    volume_scaling = 0.3
    radius_scaling = 0.7

    simulation.execute(
        beta=beta,
        alpha=alpha,
        A_eq_star_scaling=A_eq_star_scaling,
        P_star=P_star,
        radius_scaling=radius_scaling,
        volume_scaling = volume_scaling,
        # write_results=True,
        # max_reset_count=20,
        run_number=run_number,
        # write_path="f:\\Bel_Simulation\\outputs_13"
        )

    return simulation

In [14]:
simulation = instantiate_and_run_simulation(0)

In [12]:
simulation.positions

array([[ 3.91454033e-001,  8.36636678e+161, -2.48235832e-001],
       [-1.61226969e+000,  1.61686190e+126,  1.69782706e+000],
       [ 1.71704832e-002,  2.55801142e+001, -2.96398945e+000],
       [ 3.10043892e+000,  1.30377827e+046,  3.46689165e+000],
       [-4.83930216e+000,  3.56484732e+022, -1.27301830e+000],
       [ 5.06223573e+000,  9.95832623e+000, -2.56658369e+000],
       [-1.69598090e+000,  8.28527991e+000,  5.57782454e+000],
       [-2.73436655e+000,  8.08094245e+000, -5.59711968e+000],
       [ 5.47358551e+000,  6.79100472e+000,  1.62778086e+000],
       [-5.89571670e+000,  7.45848116e+000,  2.98056334e+000],
       [ 2.44583414e+000,  7.11560453e+000, -6.49702181e+000],
       [ 2.18702987e+000,  5.96119479e+000,  6.16605888e+000],
       [-5.97680407e+000,  5.70249120e+000, -3.21411168e+000],
       [ 6.79870489e+000,  5.45189459e+000, -1.69719242e+000],
       [-3.99429177e+000,  5.20549969e+000,  6.00146900e+000],
       [-1.12164676e+000,  4.96131402e+000, -7.30843125

In [5]:
simulation.results

,cluster_vol,sphericity,mean_separation,t,lumen_volume,run_no,r_min,beta,alpha,a_eq_star_scaling,p_star,mean_lifetime,lumen_volume_scaling,lumen_radius_scaling,end_reason,final_N_bodies,hull_volume
0,65.064885,0.983715,-0.039064,0.0,6.340883,0,1,106.992383,536.389191,500.0,688.827562,5,0.8,0.890903,unknown_uncaught,50,65.064885


In [8]:
np.linalg.norm(np.mean(simulation.positions, axis=0)-self.positions[-1]) 

,cluster_vol,sphericity,mean_separation,lumen_distance_from_com,t,lumen_volume,run_no,r_min,beta,alpha,a_eq_star_scaling,p_star,mean_lifetime,lumen_volume_scaling,lumen_radius_scaling,end_reason,final_N_bodies,reset_count,hull_volume
0,29.615332,0.964811,0.259214,NaN,0.00000,5.128954,0,1,148.729013,530.127447,500.0,578.400044,5,0.8,0.604576,unknown_uncaught,50,0,1176.370967
1,1176.370967,0.964811,0.259214,NaN,0.00006,5.128954,0,1,148.729013,530.127447,500.0,578.400044,5,0.8,0.604576,unknown_uncaught,50,0,1176.370967


In [7]:
def scaled_gaussian(x, mu, sigma):
    max_val = 1 / (sigma * np.sqrt(2 * np.pi))
    pdf_val = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    y = pdf_val / max_val
    return y

class ScaledSkewNormal:

    def __init__(self, a, mu, sigma):
        neg_pdf = lambda x: -skewnorm.pdf(x, a, mu, sigma)
        self.result = minimize_scalar(neg_pdf, bounds=(mu - 3*sigma, mu + 3*sigma), method='bounded')
        self.pdf_max_value = skewnorm.pdf(self.result.x, a, mu, sigma)
        self.a = a
        self.mu = mu
        self.sigma = sigma

    def get_value(self, x):
        pdf_val = skewnorm.pdf(x, self.a, self.mu, self.sigma)
        return pdf_val / self.pdf_max_value

def lumen_vol_dist(x):
    if 0 <= x <= 0.5:
        return 1
    elif 0.5 < x <= 1:
        return -2 * x + 2
    else:
        return 1e-30

def time_dist(x):
    return (x / 15) + 1e-30

def lumen_com_dist(x):
    return 1 / np.exp(0.5*x)

def derive_target(
    mean_separation: float,
    lumen_com: float,
    sphericity: float,
    n_cells: float,
    lumen_vol: float,
    hull_vol: float,
    max_time: float
) -> float:
    """
    Derive the value to be maximised from the results of the simulation.
    
    Inputs:
        results: The results of the simulation.
    
    Returns:
        The value to be maximised.
    """

    #skew_normal_volume = ScaledSkewNormal(5, -50, 500)
    skew_normal_n_cells = ScaledSkewNormal(5, -80, 500)
    skew_normal_sphericity = ScaledSkewNormal(-5, 1.05, 0.3)
    
    lumen_com_optimisation_values = lumen_com_dist(lumen_com)
    mean_separation_optimisation_value = scaled_gaussian(mean_separation, mu=-0.2, sigma=0.3)
    #volume_optimisation_value = skew_normal_n_cells.get_value(volume)
    sphericity_optimisation_value = skew_normal_sphericity.get_value(sphericity)
    n_cells_optimisation_value = skew_normal_n_cells.get_value(n_cells)
    lumen_vol_optimisation_value = lumen_vol_dist(lumen_vol/hull_vol)
    time_optimisation_value = time_dist(max_time)

    target_value = (
        mean_separation_optimisation_value *
        #volume_optimisation_value *
        sphericity_optimisation_value *
        n_cells_optimisation_value *
        lumen_vol_optimisation_value *
        time_optimisation_value *
        lumen_com_optimisation_values
    )

    return target_value

In [10]:
simulation = instantiate_and_run_simulation(1)

In [11]:
results = simulation.results.iloc[-1]
target = -derive_target(
    results['mean_separation'],
    results['lumen_distance_from_com'],
    results['sphericity'],
    results['final_N_bodies'],
    results['lumen_volume'],
    results['hull_volume'],
    results['t']
)

In [14]:
target

-0.6481651365106207

In [8]:
with sim.tqdm_joblib(tqdm(desc="My Simulation", total=1_000_000)) as progress_bar:
    Parallel(n_jobs=8)(delayed(instantiate_and_run_simulation)(j) for j in range(1_000_000))

My Simulation:   0%|          | 3895/1000000 [73:27:14<20109:49:13, 72.68s/it] c:\Users\Tom\anaconda3\envs\bel_simulation\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
My Simulation:   1%|          | 11725/1000000 [218:23:52<18408:17:10, 67.06s/it] 


KeyboardInterrupt: 

In [ ]:
results = pd.concat([pd.read_parquet(f"C:\\Users\\Tom\\Documents\\Bel PhD\\Bel_Simulation\\outputs\\{i}") for i in os.listdir("C:\\Users\\Tom\\Documents\\Bel PhD\\Bel_Simulation\\outputs") if i.endswith(".parquet")])

In [10]:
simulation = sim.Simulation(N_bodies=50, timestep_reset=1000)

In [11]:
np.linalg.norm(np.mean(simulation.positions, axis=0)-simulation.positions[-1])

0.002725328356610611

In [18]:
np.linalg.norm(np.mean(simulation.positions, axis=0)- simulation.positions[-1])

0.002725328356610611